In [1]:
# %pip install -q tfds-nightly keras tensorflow
# %pip install -q seaborn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.dates as date
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import keras
import matplotlib.dates as date
import tensorflow as tf
from datetime import datetime
import seaborn as sns

<h1> Read, split, preprocces </h1>

In [ ]:
# dataPATH = "lab_RssiPerCrownPerTime.csv"
dataPATH = "lab_window5_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window10_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window20_RssiPerCrownPerTime.csv"
# dataPATH = "lab_window30_RssiPerCrownPerTime.csv"

df = pd.read_csv(dataPATH, index_col=0)
print(df)

# features = df[['0']].copy()
features = df[['0','mean0', 'std0']].copy() # 'mean0', 'std0'

# label = df[['distance']].copy()
label = df[['distance']].copy()

X_train, X_test, y_train, y_test = train_test_split(
    features, label, test_size=0.2, random_state=1, stratify=label)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

columnames = list(features.columns)+list(label.columns)
sns.pairplot(df[columnames], diag_kind='kde')

<h1> Layered Models Build </h1>

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(64, input_dim=6, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3)  # Output layer with 3 units for (x, y, z)
])

model.compile(loss='mean_squared_error', optimizer='adam')

<h1> Training </h1>

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_split = 0.2, epochs=50, verbose = 0)

<h1> Evaluating model and Plotting </h1>

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [Distance]')
  plt.legend()
  plt.grid(True)

plot_loss(history)

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

test_results = {}

test_results['location_model'] = model.evaluate(
    X_test,
    y_test, verbose=0)

print(test_results)

In [ ]:
test_predictions = model.predict(X_test).flatten()

a = plt.axes(aspect='equal')
plt.scatter(y_test, test_predictions)
plt.xlabel('True Values [Distance]')
plt.ylabel('Predictions [Distance]')
lims = [0, 21]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
plt.show()

error = test_predictions - np.array(y_test)
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [Distance]')
_ = plt.ylabel('Count')
plt.show()

mae = mean_absolute_error(y_test, test_predictions)
mse = mean_squared_error(y_test, test_predictions)
rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

# performance metrics on the test data
print('Mean Absolute Error:', mae)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print()

model.summary()

<h1> Tensor flow lite model for Microcontrollers </h1>

In [ ]:
# Convert Keras model to TF Lite format.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

filePath = 'C:/Users/chiel/Downloads/locationModel.tflite'
# Save the TF Lite model as file
f = open(filePath, "wb")
f.write(tflite_model)
f.close()

In [ ]:
# Run inference with TensorFlow Lite
interpreter = tf.lite.Interpreter(model_path=filePath)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()
print(output_details)

input_type = input_details[0]['dtype']
input_shape = input_details[0]['shape']

# input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
input_data = np.random.uniform(-50, -99, size=input_shape).astype(np.float32)
print(input_data)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])

# Print the results of inference
print("Inference output:", output_data)